In [ ]:
!nvidia-smi

Tue May 16 00:01:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.1 MB/s eta 0:00:00


In [ ]:
# importing required libraries
import requests
import json
import torch
import os
from tqdm import tqdm
from transformers import BertTokenizerFast
from torch.utils.data import DataLoader
from transformers import BertForQuestionAnswering
from transformers import AdamW

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,ocr]

# For Colab/linux based machines:
!wget https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
!tar -xvf xpdf-tools-linux-4.04.tar.gz && sudo cp xpdf-tools-linux-4.04/bin64/pdftotext /usr/local/bin

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,ocr] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-3pd4c5ws/farm-haystack_9c20050a88d44d77b63a8366ab0d7e0b
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-3pd4c5ws/farm-haystack_9c20050a88d44d77b63a8366ab0d7e0b
  Resolved https://github.com/deepset-ai/haystack.git to commit 4c9843017c3844985c278a0754d639d8941e9034
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━

--2023-05-16 00:02:52--  https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
Resolving dl.xpdfreader.com (dl.xpdfreader.com)... 45.79.72.155
Connecting to dl.xpdfreader.com (dl.xpdfreader.com)|45.79.72.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23687259 (23M) [application/x-gzip]
Saving to: ‘xpdf-tools-linux-4.04.tar.gz’

xpdf-tools-linux-4. 100%[===================>]  22.59M  23.2MB/s    in 1.0s    

2023-05-16 00:02:53 (23.2 MB/s) - ‘xpdf-tools-linux-4.04.tar.gz’ saved [23687259/23687259]

xpdf-tools-linux-4.04/
xpdf-tools-linux-4.04/CHANGES
xpdf-tools-linux-4.04/COPYING3
xpdf-tools-linux-4.04/INSTALL
xpdf-tools-linux-4.04/ANNOUNCE
xpdf-tools-linux-4.04/bin64/
xpdf-tools-linux-4.04/bin64/pdftopng
xpdf-tools-linux-4.04/bin64/pdftohtml
xpdf-tools-linux-4.04/bin64/pdfinfo
xpdf-tools-linux-4.04/bin64/pdffonts
xpdf-tools-linux-4.04/bin64/pdfimages
xpdf-tools-linux-4.04/bin64/pdftotext
xpdf-tools-linux-4.04/bin64/pdftoppm
xpdf-tools-linux-4.04/bin64/pd

In [ ]:
!pip install pytorch-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 176 kB 30.2 MB/s 
     |████████████████████████████████| 880 kB 46.3 MB/s 
     |████████████████████████████████| 132 kB 77.1 MB/s 
     |████████████████████████████████| 10.2 MB 52.2 MB/s 
     |████████████████████████████████| 79 kB 9.0 MB/s 
     |████████████████████████████████| 140 kB 28.4 MB/s 
     |████████████████████████████████| 127 kB 63.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=34675b13dddfdc40f24acaf9dea51aca8be931caf1644ce60559e50bb805ed7c
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


# Performing Question Answering with a Fine-Tuned BERT

In [ ]:
# We have training data built using Haystack Data Annotator [https://annotate.deepset.ai/index.html]
# This file has been saved in my Google Drive
with open('/content/drive/MyDrive/train.json', 'rb') as f:
  merged= json.load(f)

In [ ]:
print('Size of training dataset ' + str(len(merged['data'])))

Size of training dataset 120


In [ ]:
# This file has been saved in my Google Drive
with open('/content/drive/MyDrive/test.json', 'rb') as f:
  val_data= json.load(f)

print('Size of validation dataset ' + str(len(val_data['data'])))

Size of validation dataset 30


In [ ]:
from haystack.nodes import FARMReader
from haystack.utils import fetch_archive_from_http

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


### epochs = 1

In [ ]:
reader = FARMReader(model_name_or_path="bert-large-uncased-whole-word-masking-finetuned-squad", use_gpu=True)
data_dir = "/content/drive/MyDrive"
reader.train(data_dir=data_dir, train_filename="merged_file.json", use_gpu=True, n_epochs=1, save_dir="MyDrive")


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'bert-large-uncased-whole-word-masking-finetuned-squad' (Bert)


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'bert-large-uncased-whole-word-masking-finetuned-squad' (Bert model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.data_handler.data_silo:
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|
 (o)(o)------'\ _ /     ( )
 
INFO:haystack.modeling.data_handler.data_silo:LOADING TRAIN DATA
INFO:haystack.modeling.data_handler.data_silo:==================
INFO:haystack.modeling.data_handler.data_silo:Loading train set from: /content/drive/MyDrive/merged_file.json 


Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:LOADING DEV DATA
INFO:haystack.modeling.data_handler.data_silo:=================
INFO:haystack.modeling.data_handler.data_silo:No dev set is being loaded
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:LOADING TEST DATA
INFO:haystack.modeling.data_handler.data_silo:=================
INFO:haystack.modeling.data_handler.data_silo:No test set is being loaded
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:DATASETS SUMMARY
INFO:haystack.modeling.data_handler.data_silo:================
INFO:haystack.modeling.data_handler.data_silo:Examples in train: 525
INFO:haystack.modeling.data_handler.data_silo:Examples in dev  : 0
INFO:haystack.modeling.data_handler.data_silo:Examples in test : 0
INFO:haystack.modeling.data_handler.data_silo:Total examples   : 525
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystac

  0%|          | 0/53 [00:00<?, ?it/s]

INFO:haystack.nodes.reader.farm:Saving reader model to devansh_model


In [ ]:
data_dir = "/content/drive/MyDrive"
results = reader.eval_on_file(data_dir, "/content/drive/MyDrive/test.json", device="cuda")
results

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
INFO:haystack.modeling.data_handler.data_silo:
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|
 (o)(o)------'\ _ /     ( )
 
INFO:haystack.modeling.data_handler.data_silo:LOADING TRAIN DATA
INFO:haystack.modeling.data_handler.data_silo:==================
INFO:haystack.modeling.data_handler.data_silo:No train set is being

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:DATASETS SUMMARY
INFO:haystack.modeling.data_handler.data_silo:================
INFO:haystack.modeling.data_handler.data_silo:Examples in train: 0
INFO:haystack.modeling.data_handler.data_silo:Examples in dev  : 0
INFO:haystack.modeling.data_handler.data_silo:Examples in test : 89
INFO:haystack.modeling.data_handler.data_silo:Total examples   : 89
INFO:haystack.modeling.data_handler.data_silo:


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

{'EM': 30.337078651685395,
 'f1': 44.376157435999964,
 'top_n_accuracy': 88.76404494382022,
 'top_n': 4,
 'EM_text_answer': 23.75,
 'f1_text_answer': 39.36847514754996,
 'top_n_accuracy_text_answer': 87.5,
 'top_n_EM_text_answer': 33.75,
 'top_n_f1_text_answer': 70.49368536713297,
 'Total_text_answer': 80,
 'EM_no_answer': 88.88888888888889,
 'f1_no_answer': 88.88888888888889,
 'top_n_accuracy_no_answer': 100.0,
 'Total_no_answer': 9}

### epochs = 10

In [ ]:
reader = FARMReader(model_name_or_path="bert-large-uncased-whole-word-masking-finetuned-squad", use_gpu=True)
data_dir = "/content/drive/MyDrive"
reader.train(data_dir=data_dir, train_filename="merged_file.json", use_gpu=True, n_epochs=10, save_dir="MyDrive")

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'bert-large-uncased-whole-word-masking-finetuned-squad' (Bert)
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'bert-large-uncased-whole-word-masking-finetuned-squad' (Bert model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 -

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:LOADING DEV DATA
INFO:haystack.modeling.data_handler.data_silo:=================
INFO:haystack.modeling.data_handler.data_silo:No dev set is being loaded
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:LOADING TEST DATA
INFO:haystack.modeling.data_handler.data_silo:=================
INFO:haystack.modeling.data_handler.data_silo:No test set is being loaded
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:DATASETS SUMMARY
INFO:haystack.modeling.data_handler.data_silo:================
INFO:haystack.modeling.data_handler.data_silo:Examples in train: 525
INFO:haystack.modeling.data_handler.data_silo:Examples in dev  : 0
INFO:haystack.modeling.data_handler.data_silo:Examples in test : 0
INFO:haystack.modeling.data_handler.data_silo:Total examples   : 525
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystac

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

...

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

INFO:haystack.nodes.reader.farm:Saving reader model to MyDrive


### Run again with training and validation split

In [ ]:
data_dir = "/content/drive/MyDrive"
results = reader.eval_on_file(data_dir, "/content/drive/MyDrive/test.json", device="cuda")
results

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
INFO:haystack.modeling.data_handler.data_silo:
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|
 (o)(o)------'\ _ /     ( )
 
INFO:haystack.modeling.data_handler.data_silo:LOADING TRAIN DATA
INFO:haystack.modeling.data_handler.data_silo:==================
INFO:haystack.modeling.data_handler.data_silo:No train set is being

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:DATASETS SUMMARY
INFO:haystack.modeling.data_handler.data_silo:================
INFO:haystack.modeling.data_handler.data_silo:Examples in train: 0
INFO:haystack.modeling.data_handler.data_silo:Examples in dev  : 0
INFO:haystack.modeling.data_handler.data_silo:Examples in test : 89
INFO:haystack.modeling.data_handler.data_silo:Total examples   : 89
INFO:haystack.modeling.data_handler.data_silo:


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

{'EM': 97.75280898876404,
 'f1': 98.99411449973249,
 'top_n_accuracy': 100.0,
 'top_n': 4,
 'EM_text_answer': 97.5,
 'f1_text_answer': 98.88095238095238,
 'top_n_accuracy_text_answer': 100.0,
 'top_n_EM_text_answer': 97.5,
 'top_n_f1_text_answer': 98.88095238095238,
 'Total_text_answer': 80,
 'EM_no_answer': 100.0,
 'f1_no_answer': 100.0,
 'top_n_accuracy_no_answer': 100.0,
 'Total_no_answer': 9}

We are able to get a 93% accuracy for our context.

Test on a Single Question

In [ ]:
import torch

In [ ]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')



## Example Question



I will now try an example question and answer.

In [ ]:
question = "What is San Francisco known for?"

In [ ]:
answer_text = "San Francisco, officially the City and County of San Francisco, is the commercial, financial, and cultural center of Northern California. The city proper is the fourth most populous in California and 17th most populous in the United States, with 815,201 residents as of 2021."

## Find out tokens in this example input

In [ ]:
input_ids = tokenizer.encode(question, answer_text)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 64 tokens.


In [ ]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)

out = []
for token, id in zip(tokens, input_ids):
    out.append('{:}'.format(token))

out
    

['[CLS]',
 'what',
 'is',
 'san',
 'francisco',
 'known',
 'for',
 '?',
 '[SEP]',
 'san',
 'francisco',
 ',',
 'officially',
 'the',
 'city',
 'and',
 'county',
 'of',
 'san',
 'francisco',
 ',',
 'is',
 'the',
 'commercial',
 ',',
 'financial',
 ',',
 'and',
 'cultural',
 'center',
 'of',
 'northern',
 'california',
 '.',
 'the',
 'city',
 'proper',
 'is',
 'the',
 'fourth',
 'most',
 'populous',
 'in',
 'california',
 'and',
 '17th',
 'most',
 'populous',
 'in',
 'the',
 'united',
 'states',
 ',',
 'with',
 '81',
 '##5',
 ',',
 '201',
 'residents',
 'as',
 'of',
 '2021',
 '.',
 '[SEP]']

In [ ]:
# run the model with the inputs
outputs = model(torch.tensor([input_ids]),
                             return_dict=True) 

start_scores = outputs.start_logits
end_scores = outputs.end_logits


Now we can highlight the answer just by looking at the most probable start and end words. 

In [ ]:
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

Answer: "commercial , financial , and cultural center of northern california"


Answer is correct. This shows to be very powerful.

# Conclusion

In [ ]:
start_scores = start_scores.detach().numpy().flatten()
end_scores = end_scores.detach().numpy().flatten()

Score distribution for each word in the context

In [ ]:
import pandas as pd

scores = []
for (i, token_label) in enumerate(out):

    scores.append({'token_label': token_label, 
                   'score': start_scores[i],
                   'Type': 'start'})
    
    scores.append({'token_label': token_label, 
                   'score': end_scores[i],
                   'Type': 'end'})
    
df = pd.DataFrame(scores)
